# Feature extraction with lexical mining
## R and Python community characteristics



In [1]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import re
import pickle 

import matplotlib.pyplot as plt 
import seaborn as sns

Once again we will work with the scraped data from the data science and stats stackexchanges and stack overflow. These are the same data we used last time. The code below extracts the relevant columns. Like last time most of the exercises can be completed by applying the exact same functions to each of the datasets. 

Run the code block below to get all of the data you need (you might first want to move the csv files from last time into your working directory)

In [2]:
# LOAD DATA
# Load overflow data
overflow_df = pd.read_csv('Posts_overflow.csv').dropna(subset = ['Body'])
# Load the datascience data. 
datascience_df = pd.read_csv('Posts_ds.csv').dropna(subset = ['Body'])
# load stats data
stats_df = pd.read_csv('Posts_stats.csv').dropna(subset = ['Body'])




def extract_from_to_column(df, regex, from_col, to_col):
    df[to_col] = df[from_col].apply(lambda x: ' '.join(regex.findall(x)))    
    return df

code_str = '<pre><code>.+?</code></pre>'
code_regex = re.compile(code_str, flags = re.DOTALL|re.UNICODE)


# Then we remove the code from the html body
def remove_code_from_body(df, regex):
    df['Body'] = df.Body.apply(lambda x: re.sub(regex, ' ', x))
    return df 


comment_str = '#.*$'
comment_regex = re.compile(comment_str, flags = re.DOTALL|re.UNICODE)

def get_text(html):
    soup = BeautifulSoup(html, 'lxml')
    text = soup.get_text()
    return text

def extract_text(df):    
    df['text'] = df.Body.apply(lambda x: get_text(x))    
    return df 


tag_str = '(?<=<).*?(?=>)'
tag_re = re.compile(tag_str, flags = re.DOTALL|re.UNICODE)


def tags_if_any(x, regex):
    if pd.isna(x):
        return list()
    return regex.findall(x)

def find_tags(df, regex):
    df['tags_l'] = df.Tags.apply(lambda x: tags_if_any(x, regex))
    return df 



overflow_df    = extract_from_to_column(overflow_df, code_regex, from_col = 'Body', to_col = 'code')
datascience_df = extract_from_to_column(datascience_df, code_regex, from_col = 'Body', to_col = 'code')
stats_df       = extract_from_to_column(stats_df, code_regex, from_col = 'Body', to_col = 'code')

overflow_df    = remove_code_from_body(overflow_df, code_regex)
datascience_df = remove_code_from_body(datascience_df, code_regex)
stats_df       = remove_code_from_body(stats_df, code_regex)

overflow_df = extract_from_to_column(overflow_df, comment_regex, from_col = 'code', to_col = 'comments')
datascience_df = extract_from_to_column(datascience_df, comment_regex, from_col = 'code', to_col = 'comments')
stats_df = extract_from_to_column(stats_df, comment_regex, from_col = 'code', to_col = 'comments')

overflow_df    = extract_text(overflow_df)
datascience_df = extract_text(datascience_df)
stats_df       = extract_text(stats_df)

overflow_df    = find_tags(overflow_df, tag_re).sample(20000)
datascience_df = find_tags(datascience_df, tag_re).sample(20000)
stats_df       = find_tags(stats_df, tag_re).sample(20000)


## Exercise 12.1: A simple manual lexicon approach

This assignment is intended to give you a clearer understanding of what lexical approaches are, give you some intuition for how tokenization works.

**Exercise 12.1.1** before we move on to prebuild lexicons we will construct our own simple example. Begin by defining two dictionaries with keys corresponding to the words you want to search for and a value of `1` for each key (we will simply count the number of occurences of our words). An example of lexica could be 

```python
py_lexicon = {'python': 1,
              'py': 1,
              'ipy':1}

r_lexicon = {'r': 1,
             'tidyverse': 1,
             'tidy':1}
```

Create a new column in each of the dataset which contains a list of each word in the `text` column (hint: remember to lower/upper all characters here, to match both _'Python'_ and _'python'_. Is this a good way to tokenize the text? Could you come up with improvements?



In [3]:
# [Answer to ex. 12.1.1 here]

**Ex 12.1.2** Count for each observation in the datasets how many times one of the words from each of the lexica occurs in the text and store the results in two new columns called `py_hits` and `r_hits`.

In [4]:
# [Answer to ex. 12.1.2 here]

**Ex. 12.1.3** Show the joint distribution of hits in the `r` and `py` lexica by plotting them in a scatter plot. You should plot a separate scatter for each of the forums (datasets). 

> _Hint:_ Use the following function to give the points a slight randomness in position. This is visually appealing when looking at  discrete data, and give some insight into the density.
>
> ```python
> def rand_jitter(arr):
>    stdev = .01*(max(arr)-min(arr))
>    return arr + np.random.randn(len(arr)) * stdev
> ```

In [5]:
# [Answer to ex. 12.1.3 here]

## Exercise 12.2 - Using different dictionaries

In this exercise we will make two improvements over the stuff we did above. First of all we will use a proper tokenizer instead of the simple `str.split` approach, and we will work with a number of different lexicons that map out different aspect of a text. 

**Ex 12.2.1** Import nltk and define a new function that takes a string as input and tokenizes it using `nltk.word_tokenize`. Also call `str.lower` on all tokens before returning a list. Test the function on an example from one of the datasets to see that it works.

In [6]:
# [Answer to ex. 12.2.1 here]

**Ex. 12.2.2** Compare the nltk tokenizer to the simple `str.split` method you implemented above. What are the differences? Which one do you prefer? 

> _Note:_ the choice of tokenizer is not always obvious, for example some preserve emojis while others do not. Understand the text you are dealing with and choosing a tokenizer accordingly.


In [7]:
# [Answer to ex. 12.2.2 here]

**Ex 12.2.3** Some lexical scorers take raw strings as inputs while others take tokens stored in a list as input. The first scorer you should try is Afinn which requires a string as input. 

Import Afinn and set up a sentiment scorer. Show the distribution of sentiments separately for all posts containing the tag `r` and `python`. Do this for each of the three datasets. 

Explain intuitively what Afinn does - what does it mean that there is non-zero density at +10 in the graph you just plotted? 

In [8]:
# [Answer to ex. 12.1.3 here]

**Ex. 12.2.4** Afinn is a fairly simple lexical approach, next you should try out a slightly more complex method. Load the _argument_ dictionary by running the line (after downloading text2arg.pkl from Absalon and placing it in the same folder as your notebook.)
```python
class2re,string_test = pickle.load(open('text2arg.pkl','rb'))
```
You need to do a little bit of coding to get scores from the lexicon. `class2re` is a dictionary where keys are a specific type of argumentation and the values are regex'es that pick up this specific argumentation trait. Write a function that loops through the key-value pairs of class2re (_Hint:_ `dict.items()`). Use the `re.findall` method to get all matches for a specific text input. (ask if in doubt about how to do this)

Finally test out the function on the `string_test` snippet.

In [9]:
# [Answer to ex. 12.2.4 here]

**Ex.12.2.5** apply the arguments regexes to the data containing the either an `r` or `python` tag. Show the density of the _causation_ score in each of the three datasets

In [10]:
# [Answer to ex. 12.2.5 here]

**Ex. 12.2.6 (BONUS)** The last approach you will try is a mix of sentiment/topic lexicons from multiple source. Load the raw data by running
```python
w2conglomerate,conglomerate_cols = pickle.load(open('conglomerate.pkl','rb'))
```
Try to make sense of the contents of `w2conglomerate` and `conglomerate_cols`. Test the lexicon on the `string_test` you were given in ex. 1.2.4. Then apply the lexicon to get the _joy_ score of `r` and `python` tagged posts across all three datasets. 

In [11]:
# [Answer to ex. 12.2.6 here]